In [1]:
import pandas as pd
import numpy as np
from surprise import SVD, accuracy
from surprise import Reader, Dataset

In [2]:
def query_postgreSQL(query):
     import pandas as pd
     import psycopg2 as pg
     from datetime import datetime
    
  

    # DB Connection
     conn = pg.connect(host="postgresql-feidear.alwaysdata.net", dbname="feidear_data", user="feidear_ji",
                            port="5432",
                            password="feidear1234")
    #  start_tm = datetime.now()

     # Get a DataFrame

     query_result = pd.read_sql(query, conn) # type: ignore 
     # Close connection
     conn.close()
 
     return query_result
query = """
        SELECT *
            FROM mldataset
    """

rating = query_postgreSQL(query) # mldataset DB에 있는 정보를 dataframe으로 바꿈.
rating = pd.DataFrame(rating, columns=["u_id","r_fetv_name","r_fetv_est"])

rating.head()

c:\Users\kosmo\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,u_id,r_fetv_name,r_fetv_est
0,jungin,종로한복축제,4
1,jungin,제주왕벚꽃축제,2.5
2,jungin,"국립생태원 반려식물 특별전(나와 지구, 모두의 힐링)",4.3
3,jungin,2022 제4회 양재플라워페스타,2.5
4,jungin,태안 백합꽃 정원,2.5


In [3]:
rating['u_id'].value_counts()
rating['r_fetv_name'].value_counts()

# 평가한 적 없는 축제 가려냄
tab=pd.crosstab(rating['u_id'], rating['r_fetv_name'])
tab

r_fetv_name,2022 제4회 양재플라워페스타,"국립생태원 반려식물 특별전(나와 지구, 모두의 힐링)",마티네 콘서트<마술피리>,부산자갈치축제,이월드 벚꽃축제 : 블라썸 피크닉,장군산 영평사 구절초꽃축제,제주왕벚꽃축제,종로한복축제,태안 백합꽃 정원,팔공산 벚꽃축제
u_id,,,,,,,,,,
jungin,1,1,0,0,0,0,1,1,1,0
one1,0,0,1,1,1,1,0,0,0,1


In [4]:
# 두 개의 집단변수를 가지고 나머지 rating을 그룹화
rating_g = rating.groupby(['u_id', 'r_fetv_name'])
rating_g.sum()
tab = rating_g.sum().unstack() # 행렬구조로 변환
tab

r_fetv_est                                              \
r_fetv_name 2022 제4회 양재플라워페스타 국립생태원 반려식물 특별전(나와 지구, 모두의 힐링) 마티네 콘서트<마술피리>   
u_id                                                                        
jungin                    2.5                           4.3           NaN   
one1                      NaN                           NaN           1.8   

                                                                        \
r_fetv_name 부산자갈치축제 이월드 벚꽃축제 : 블라썸 피크닉 장군산 영평사 구절초꽃축제  제주왕벚꽃축제  종로한복축제   
u_id                                                                     
jungin          NaN                NaN             NaN      2.5      4   
one1              1                  5               5      NaN    NaN   

                                  
r_fetv_name 태안 백합꽃 정원  팔공산 벚꽃축제   
u_id                              
jungin             2.5       NaN  
one1               NaN         5

In [18]:
temp = tab.loc['jungin'][ tab.loc['jungin'].isna() ]
# print(temp)

'''
temp = tab[tab.isna()].loc['jungin'] # type: ignore
print(temp)
print('-'*100)
'''
temp = temp.index.to_list()
# print(temp)
# print('-'*100)

res=[]
for t in temp :
    res.append(t[-1])
res    

            r_fetv_name       
r_fetv_est  마티네 콘서트<마술피리>         NaN
            부산자갈치축제               NaN
            이월드 벚꽃축제 : 블라썸 피크닉    NaN
            장군산 영평사 구절초꽃축제        NaN
            팔공산 벚꽃축제              NaN
Name: jungin, dtype: object
[('r_fetv_est', '마티네 콘서트<마술피리>'), ('r_fetv_est', '부산자갈치축제'), ('r_fetv_est', '이월드 벚꽃축제 : 블라썸 피크닉'), ('r_fetv_est', '장군산 영평사 구절초꽃축제 '), ('r_fetv_est', '팔공산 벚꽃축제 ')]
----------------------------------------------------------------------------------------------------


['마티네 콘서트<마술피리>',
 '부산자갈치축제',
 '이월드 벚꽃축제 : 블라썸 피크닉',
 '장군산 영평사 구절초꽃축제 ',
 '팔공산 벚꽃축제 ']

In [1]:
#rating 데이터셋 생성
reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(df=rating, reader=reader)

data

NameError: name 'Reader' is not defined

In [20]:
train = data.build_full_trainset() # 훈련셋
test = train.build_testset() # 검정셋

In [21]:
# help(SVD)
# 모델 생성
model = SVD(n_factors=100, n_epochs=20, random_state=123)
model.fit(train) # model 생성

In [22]:
# jungin 사용자에게 축제 추천
user_id = 'jungin' # 추천대상자
item_ids = res # 추천 대상 축제 리스트
actual_rating = 0 # 실제 평점

result = []
for item_id in item_ids :
    predict = model.predict(user_id, item_id, actual_rating)
    #print(predict)
    
    #pred = np.sort(predict, axis=0)
    #print(pred)
    temp = [predict.uid, predict.iid, predict.est]
    result.append(temp)
result.sort(key=lambda x:(x[0], x[1]), reverse=True)

num = 1
for r in result[:3] :
    print('Top',num,'::',r[1])
    num+=1

Top 1 :: 팔공산 벚꽃축제 
Top 2 :: 장군산 영평사 구절초꽃축제 
Top 3 :: 이월드 벚꽃축제 : 블라썸 피크닉
